In [1]:
import ee
import geemap
import pandas as pd

# Authenticate the EE API
ee.Authenticate()

# Initialize EE API
ee.Initialize(project='amiropenbuilding')

# Loading required adminBoundaries, admin0: country, admin1: province, admin2: city
admin0 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM0")
admin1 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM1")
admin2 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2")

# Loading openBuildings dataSet
openBuildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")

# m= geemap.Map()
# m

In [3]:
# # List of countryNames
# admin0Name= admin0.aggregate_array('shapeName')
# admin0Name_list = admin0Name.getInfo()

# # List of countryAbbr.
# admin0Abbr= admin0.aggregate_array('shapeGroup')
# admin0Abbr_list = admin0Abbr.getInfo()

# # Print the list of countryNames and countryAbbr.
# print(sorted(admin0Name_list))
# print(sorted(admin0Abbr_list))

# print(f"There are worldwide {len(admin0Name_list)} countries in the admin0 level.")

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia (Plurinational State *', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'Colombia', 'Comoros', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', "Côte d'Ivoire", 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Federal Democratic Republic of Nepal', 'Federated States of Micronesia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Greenland', 'Grenada', 'Guatemala', 'Guinea', 'Guin

In [4]:
# # List of provinceNames for the selected country
# admin1Name= admin1.aggregate_array('shapeName')
# admin1Name_list = admin1Name.getInfo()

# # Print the Name of all Countries WorldWide
# print(f"There are worldwide {len(admin1Name_list):,} provinces in the admin1 level.")

There are worldwide 3,291 provinces in the admin1 level.


In [5]:
# # List of provinceNames
# admin2Name= admin2.aggregate_array('shapeName')
# admin2Name_list = admin2Name.getInfo()

# # Total number of the list of countryNames
# print(f"There are worldwide {len(admin2Name_list):,} cities in the admin2 level.")

There are worldwide 113,035 cities in the admin2 level.


#### At this step, the user specifies Area Of Iterest (AOI)

In [3]:
# User specifies country of interest
theSelectedCountry= input("Please Enter Country Name:")
print(f"You selected: {theSelectedCountry}")

# Search for Abbr. of te selected country
countryAbbr = admin0.filter(ee.Filter.eq("shapeName" , theSelectedCountry)).first().get("shapeGroup").getInfo()
print(f"The Abbr. of {theSelectedCountry} is: {countryAbbr}")

# Generates list of the provinces in the selected country
theCountryProvinces = admin1.filter(ee.Filter.eq("shapeGroup",countryAbbr)).aggregate_array("shapeName").getInfo()

# Printing the province Names in table  with row numbers
print(f"\nTere are {len(theCountryProvinces)} provinces/federal states in {theSelectedCountry}, they are:\n")
for i, province in enumerate(theCountryProvinces, start=1):
    print(f"{i}. {province}")

# User specifies the province of Interest
theSelectedProvince = input(f"Please Enter Provice of Interest:")
print(f"\nYou selected {theSelectedProvince}")

You selected: Ecuador
The Abbr. of Ecuador is: ECU

Tere are 25 provinces/federal states in Ecuador, they are:

1. ZONA NO DELIMITADA
2. AZUAY
3. BOLIVAR
4. CAÑAR
5. CARCHI
6. COTOPAXI
7. CHIMBORAZO
8. EL ORO
9. ESMERALDAS
10. GUAYAS
11. IMBABURA
12. LOJA
13. LOS RIOS
14. MANABI
15. MORONA SANTIAGO
16. NAPO
17. PASTAZA
18. PICHINCHA
19. TUNGURAHUA
20. ZAMORA CHINCHIPE
21. GALAPAGOS
22. SUCUMBIOS
23. ORELLANA
24. SANTO DOMINGO DE LOS TSACHILAS
25. SANTA ELENA

You selected PASTAZA


In [4]:
theSelectedProvinceShapeID = admin1.filter(ee.Filter.eq("shapeName",theSelectedProvince)).first().get("shapeID").getInfo()
print(theSelectedProvinceShapeID)

theSelectedProvinceCities = admin2.filter(ee.Filter.eq("ADM1_shape",theSelectedProvinceShapeID)).aggregate_array("shapeName").getInfo()
# Printing the province Names in table  with row numbers
print(f"\nThere are {len(theSelectedProvinceCities)} cities in {theSelectedProvince} and are:\n")
for i, city in enumerate(theSelectedProvinceCities, start=1):
    print(f"{i}. {city}")

ECU-ADM1-3_0_0-B16

There are 4 cities in PASTAZA and are:

1. Cantón Santa Clara
2. Cantón Mera
3. Pastaza
4. Cantón Arajuno


In [5]:
theSelectedCityName = input("Please Enter Name of the City of Interest:")
geometry0 = admin0.filter(ee.Filter.eq("shapeName",theSelectedCountry)).geometry()
geometry1 = admin1.filter(ee.Filter.eq("shapeName",theSelectedProvince)).geometry()
geometry2 = admin2.filter(ee.Filter.eq("shapeName",theSelectedCityName)).geometry()

In [6]:
Map = geemap.Map()
Map.centerObject(geometry2,8)
Map.addLayer(geometry2, {'color': 'red'}, 'theSelectedCity')
Map

Map(center=[-1.8974646147383345, -76.88307699177732], controls=(WidgetControl(options=['position', 'transparen…

In [7]:
filteredBuildings = openBuildings.filter(ee.Filter.bounds(geometry2))

# Count the number of building footprints in the filtered collection
buildingCount = filteredBuildings.size().getInfo()

# Print the result
print(f"The number of building footprints in {theSelectedCityName} is: {buildingCount:,}")

The number of building footprints in Pastaza is: 30,217


In [8]:
import pandas as pd
import json

# Export the filtered data to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=filteredBuildings,
    description= f'filtered_buildings_{theSelectedCityName}',
    fileNamePrefix= f'filtered_buildings_{theSelectedCityName}',
    fileFormat='CSV'
)

task.start()

In [9]:
import gdown

# Google Drive file ID from the shareable link
file_id = '1NLJUD6sW5C5X0buTD2AUBm3Zp2tZKnjm'
url = f'https://drive.google.com/uc?id={file_id}'

output = 'filtered_buildings.csv'
gdown.download(url, output, quiet=False)

df = pd.read_csv(output)

# Print the first few rows of the DataFrame
df.head()



Downloading...
From: https://drive.google.com/uc?id=1NLJUD6sW5C5X0buTD2AUBm3Zp2tZKnjm
To: e:\2023\SCOR\myPythonFiles\filtered_buildings.csv
100%|██████████| 12.1M/12.1M [00:32<00:00, 372kB/s]


,system:index,area_in_meters,confidence,full_plus_code,longitude_latitude,.geo
0,10d0000000000002c698,21.437099,0.6875,6795F564+GQP5,"{""type"":""Point"",""coordinates"":[-76.84303742527...","{""type"":""Polygon"",""coordinates"":[[[-76.8430083..."
1,10d0000000000004599a,53.526100,0.6875,67C4853X+WQR6,"{""type"":""Point"",""coordinates"":[-77.80062045830...","{""type"":""Polygon"",""coordinates"":[[[-77.8005698..."
2,10d00000000000008804,33.641300,0.7500,67C4966M+WQVM,"{""type"":""Point"",""coordinates"":[-77.76558084222...","{""type"":""Polygon"",""coordinates"":[[[-77.7655606..."
3,10d000000000000093e7,54.719898,0.7500,67C483FW+GWGX,"{""type"":""Point"",""coordinates"":[-77.90266252681...","{""type"":""Polygon"",""coordinates"":[[[-77.9026206..."
4,10d0000000000000dfc7,64.352997,0.7500,67C476CQ+VGPP,"{""type"":""Point"",""coordinates"":[-77.76117079128...","{""type"":""Polygon"",""coordinates"":[[[-77.7611201..."


In [10]:
Map.addLayer(filteredBuildings, {}, " openBuildings")
Map

Map(bottom=33413.0, center=[-1.8974646147383345, -76.88307699177732], controls=(WidgetControl(options=['positi…

In [42]:
dsm = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").select('DSM')
dsm1 =dsm.median()
dsmBounded = dsm1.clip(geometry2)
elevationVisDsm = {'min': 0, 'max': 5000, 'palette':['0000ff','00ffff','ffff00','ff0000','ffffff']}

dtm = ee.Image("USGS/GMTED2010_FULL").select('be75')
dtmBounded = dtm.clip(geometry2)
elevationVisDtm = {'min': -100, 'max': 6500, 'gamma': 3.5}

# Map.addLayer(dsmBounded , elevationVisDsm, "DSM")
# Map.addLayer(dtmBounded , elevationVisDtm, "DTM")
subVis = {'min': -5000, 'max': 5000, 'palette':['0000ff','00ffff','ffff00','ff0000','ffffff']}
chm = dsm1.subtract(dtm) 
# Map.addLayer(chm.clip(geometry2) , subVis, "CHM")
# Map


In [41]:
type(chm)

ee.image.Image

In [45]:
resolutionDSM = dsm.first().projection().nominalScale()
resolutionDSM

In [46]:
resolutionDTM = dtm.projection().nominalScale()
resolutionDTM

In [69]:

def extract_dsm(feature):
    # Get the geometry of the feature
    geom = feature.geometry()
    
    # Reduce the DSM collection by calculating the mean value within the feature geometry
    dsm_mean = dsm.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale= resolutionDSM,  # Adjust scale according to the DSM data
        maxPixels=1e10,
        tileScale = 16
    )
    
    # Add the DSM mean value to the feature properties
    return feature.set(dsm_mean)

In [70]:
def extract_dtm(feature):
    # Get the geometry of the feature
    geom = feature.geometry()
    
    # Reduce the DSM collection by calculating the mean value within the feature geometry
    dtm_mean = dtm.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale= resolutionDSM,  # Adjust scale according to the DSM data
        maxPixels=1e10,
        tileScale = 16
    )
    dtm_value = dtm_mean.get('be75');
    # Add the DTM mean value to the feature properties
    return feature.set("dtm", dtm_value)

In [71]:
openBuildingsWith_dsm = filteredBuildings.map(extract_dsm)



In [72]:
openBuildingsWith_dsmdtm = openBuildingsWith_dsm.map(extract_dtm)


In [73]:
# Export the results to Google Drive as a CSV file
task = ee.batch.Export.table.toDrive(
    collection = openBuildingsWith_dsmdtm,
    description ='buildings_with_dsmdtm',
    fileNamePrefix = 'buildings_with_dsmdtm',
    fileFormat='CSV'
)

task.start()